In [ ]:
from tkinter import filedialog
import matplotlib.pyplot as plt
import pandas as pd
from decimal import *
import numpy as np
import io
import math
import random
import operator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/BBNI.csv")

In [ ]:
data = pd.DataFrame(df, columns = ['Date ;', 'open;', 'high;', 'close;', 'adj close;', 'colume'])

In [ ]:
print('SAHAM : \n')
print(data)

print('POPULASI : \n')
winSize = 4
genSize = 10
populasiSize = len(data)
populasi = []

for i in range(populasiSize):
    individu = {}
    for j in range(genSize):
        individu['theta'+str(j)] = random.random()
    populasi.append(individu)

dataKoefsiensi = pd.DataFrame(list(populasi))
print(dataKoefsiensi)

SAHAM : 

        Date ;   open;   high;  close;  adj close;           colume
0   2020-04-06  4060.0  4300.0  4290.0      4290.0   61479800;;;;;;
1   2020-04-07  4490.0  4650.0  4310.0      4310.0   81628300;;;;;;
2   2020-04-08  4200.0  4240.0  4010.0      4010.0   61532700;;;;;;
3   2020-04-09  4100.0  4150.0  3990.0      3990.0   36659600;;;;;;
4   2020-04-13  4000.0  4070.0  3990.0      3990.0   20521300;;;;;;
..         ...     ...     ...     ...         ...              ...
95  2020-09-01  5150.0  5175.0  5175.0      5175.0   54441900;;;;;;
96  2020-09-02  5250.0  5275.0  5225.0      5225.0   46207400;;;;;;
97  2020-09-03  5275.0  5375.0  5350.0      5350.0  133938600;;;;;;
98  2020-09-04  5250.0  5250.0  5225.0      5225.0   94096000;;;;;;
99  2020-09-07  5225.0  5275.0  5250.0      5250.0   40928200;;;;;;

[100 rows x 6 columns]
POPULASI : 

      theta0    theta1    theta2  ...    theta7    theta8    theta9
0   0.653419  0.892769  0.379059  ...  0.902927  0.009785  0.178950
1

In [ ]:
print('FITNESS: \n')
def MLR(idxInduvidu, hari1):
    MLR = 0
    harike = hari1

    for i in range(winSize):
        MLR = MLR + dataKoefsiensi.iat[idxInduvidu, i] * data.at[harike, 'open;']
        harike += 1
    
    return MLR

def RMSE(idxInduvidu):
    panjang = len(dataKoefsiensi) - winSize
    totalSqureEror = 0

    for x in range(panjang):
        totalSqureEror += np.power(MLR(idxInduvidu, x) - data.at[x+winSize, 'open;'], 2)
    rmse = math.sqrt(totalSqureEror/len(dataKoefsiensi)-winSize)
    return rmse

fitness_individu = []
panjang = len(dataKoefsiensi)

for i in range(panjang):
    fitness = 1 / (RMSE(i))
    fitness_individu.append(fitness)

defInduvidu = dataKoefsiensi.assign(fitness = fitness_individu)
print(defInduvidu)

FITNESS: 

      theta0    theta1    theta2  ...    theta8    theta9   fitness
0   0.653419  0.892769  0.379059  ...  0.009785  0.178950  0.000154
1   0.102462  0.125410  0.562965  ...  0.812601  0.033943  0.000603
2   0.360180  0.770923  0.937566  ...  0.635103  0.146181  0.000206
3   0.596925  0.498314  0.002530  ...  0.846648  0.945157  0.000220
4   0.033291  0.346100  0.290558  ...  0.310227  0.536149  0.001326
..       ...       ...       ...  ...       ...       ...       ...
95  0.997824  0.219942  0.157127  ...  0.570312  0.329905  0.000380
96  0.833751  0.452899  0.156001  ...  0.288181  0.602010  0.000331
97  0.886955  0.076464  0.083607  ...  0.755421  0.192459  0.002976
98  0.731296  0.812424  0.075864  ...  0.662782  0.599353  0.000345
99  0.359340  0.711202  0.575809  ...  0.177402  0.491008  0.000152

[100 rows x 11 columns]


In [ ]:
print('PROBABILITAS INDIVIDU    : \n')
tot_fitness = defInduvidu['fitness'].sum()
prob = []

for i in range(len(defInduvidu)):
    nilai_prob = defInduvidu.at[i, 'fitness'] / tot_fitness
    prob.append(nilai_prob)

defInduvidu = defInduvidu.assign(probabilitas = prob)
print(defInduvidu)

PROBABILITAS INDIVIDU    : 

      theta0    theta1    theta2  ...    theta9   fitness  probabilitas
0   0.653419  0.892769  0.379059  ...  0.178950  0.000154      0.004061
1   0.102462  0.125410  0.562965  ...  0.033943  0.000603      0.015928
2   0.360180  0.770923  0.937566  ...  0.146181  0.000206      0.005446
3   0.596925  0.498314  0.002530  ...  0.945157  0.000220      0.005823
4   0.033291  0.346100  0.290558  ...  0.536149  0.001326      0.035057
..       ...       ...       ...  ...       ...       ...           ...
95  0.997824  0.219942  0.157127  ...  0.329905  0.000380      0.010054
96  0.833751  0.452899  0.156001  ...  0.602010  0.000331      0.008755
97  0.886955  0.076464  0.083607  ...  0.192459  0.002976      0.078663
98  0.731296  0.812424  0.075864  ...  0.599353  0.000345      0.009115
99  0.359340  0.711202  0.575809  ...  0.491008  0.000152      0.004023

[100 rows x 12 columns]


In [ ]:
print('MEMILIH INDIVIDU NEW GEN  : \n')
def select_individu(df_populasi):
    rand = random.random()
    temp_sum = 0

    for i in range(len(df_populasi)):
        temp_sum += df_populasi.at[i, 'probabilitas']

        if temp_sum > rand:
            return i

selected = []

for i in range(len(defInduvidu)):
    select = select_individu(defInduvidu)
    selected.append(select)

listSelect = pd.Series(selected)

newGen = []

for i in range(len(listSelect)):
    newGen.append(defInduvidu.iloc[listSelect[i]])

newGen = pd.DataFrame(newGen).reset_index(drop=True)
print(newGen)

MEMILIH INDIVIDU NEW GEN  : 

      theta0    theta1    theta2  ...    theta9   fitness  probabilitas
0   0.309969  0.913147  0.919596  ...  0.318110  0.000108      0.002862
1   0.919555  0.412786  0.166173  ...  0.629480  0.000260      0.006872
2   0.132635  0.299089  0.376989  ...  0.851855  0.003024      0.079942
3   0.646581  0.237730  0.012016  ...  0.272910  0.002825      0.074671
4   0.833751  0.452899  0.156001  ...  0.602010  0.000331      0.008755
..       ...       ...       ...  ...       ...       ...           ...
95  0.033291  0.346100  0.290558  ...  0.536149  0.001326      0.035057
96  0.123186  0.343605  0.178686  ...  0.533307  0.000660      0.017453
97  0.591154  0.842722  0.162143  ...  0.051703  0.000155      0.004096
98  0.897971  0.823544  0.012505  ...  0.192861  0.000258      0.006823
99  0.462462  0.305234  0.266084  ...  0.223435  0.000429      0.011345

[100 rows x 12 columns]


In [ ]:
uk_tour = 5

def tournamentSelection(df_popopulasi, uk_tour):
    best_krom = []
    for i in range(1, uk_tour):
        krom = pop[random.randint(0,len(pop)-1)]
        if (best_krom == [] or hitungFitness(krom) > hitungFitness(best_krom)):
            best_krom = krom
    return best_krom

In [ ]:
print('CROSSOVER    : \n')
list_offspring = []

konstanta = 0.5
cr = 0.5

def crossover(p1, p2, list_offspring):

    offspring1 = {}
    for i in range(winSize+1):
        offspring1['theta' + str(i)] = konstanta*p1.iat[i] + (1-konstanta)*p2.iat[i]
        list_offspring.append(offspring1)

    offspring2 = {}
    for i in range(winSize+1):
        offspring2['theta'+str(i)] = konstanta*p2.iat[i] + (1-konstanta)*p1.iat[i]
        list_offspring.append(offspring2)

sum_parent = int(len(newGen)*cr)

for i in range(sum_parent):
    if ((i+1)%2) ==1:
        parent1 = newGen.iloc[i]
    if ((i+1)%2)==0:
        parent2 = newGen.iloc[i]
        crossover(parent1, parent2, list_offspring)
    
anak = pd.DataFrame(list_offspring)

newGen = newGen.drop('fitness', axis=1)
newGen = newGen.drop('probabilitas', axis=1)
newGen.update(anak)

print(newGen)

CROSSOVER    : 

      theta0    theta1    theta2  ...    theta7    theta8    theta9
0   0.614762  0.662966  0.542884  ...  0.418753  0.139230  0.318110
1   0.614762  0.662966  0.542884  ...  0.130684  0.985397  0.629480
2   0.614762  0.662966  0.542884  ...  0.225573  0.296380  0.851855
3   0.614762  0.662966  0.542884  ...  0.247692  0.813164  0.272910
4   0.614762  0.662966  0.542884  ...  0.872795  0.288181  0.602010
..       ...       ...       ...  ...       ...       ...       ...
95  0.413512  0.500878  0.123901  ...  0.681758  0.310227  0.536149
96  0.413512  0.500878  0.123901  ...  0.737160  0.080581  0.533307
97  0.413512  0.500878  0.123901  ...  0.330749  0.170033  0.051703
98  0.413512  0.500878  0.123901  ...  0.507390  0.617117  0.192861
99  0.413512  0.500878  0.123901  ...  0.878782  0.555313  0.223435

[100 rows x 10 columns]


In [ ]:
print('MUTASI DAN PERGANTIAN GENERASI BARU   : \n.')
mr = 0.8
selected_mutant = int(populasiSize*mr)

idxTerpilih = []
for i in range(selected_mutant):
    rand = random.randint(0, len(newGen)-1)
    if (i) == 0:
        idxTerpilih.append(rand)
    else:
        while rand in idxTerpilih:
            rand = random.randint(0, len(newGen)-1)
        idxTerpilih.append(rand)

for i in range(len(idxTerpilih)):
    igen_change = random.randint(0, genSize-1)

    perubahanGenr = random.uniform(0.2, 1.2)

    perubahanGen = newGen.iloc[idxTerpilih[i], igen_change] * perubahanGenr

    newGen.iloc[idxTerpilih[i], igen_change] = perubahanGen

print(newGen)

MUTASI DAN PERGANTIAN GENERASI BARU   : 
.
      theta0    theta1    theta2  ...    theta7    theta8    theta9
0   0.614762  0.662966  0.542884  ...  0.230781  0.139230  0.318110
1   0.614762  0.662966  0.169479  ...  0.130684  0.985397  0.629480
2   0.614762  0.662966  0.542884  ...  0.225573  0.296380  0.851855
3   0.614762  0.662966  0.542884  ...  0.247692  0.813164  0.272910
4   0.614762  0.176586  0.542884  ...  0.872795  0.288181  0.602010
..       ...       ...       ...  ...       ...       ...       ...
95  0.413512  0.392478  0.123901  ...  0.681758  0.310227  0.536149
96  0.413512  0.500878  0.123901  ...  0.737160  0.080581  0.533307
97  0.413512  0.500878  0.123901  ...  0.330749  0.103730  0.051703
98  0.413512  0.500878  0.123901  ...  0.507390  0.617117  0.192861
99  0.413512  0.500878  0.123901  ...  0.966980  0.555313  0.223435

[100 rows x 10 columns]


In [ ]:
print('UPDATE NEW GEN : \n')
fitness_individu = []

panjang = len(dataKoefsiensi)

for i in range(panjang):
    fitness = 1/(RMSE(i))
    fitness_individu.append(fitness)

newGen = newGen.assign(fitness = fitness_individu)
defInduvidu = defInduvidu.drop(['probabilitas'], axis=1)
defInduvidu = defInduvidu.append(newGen)
defInduvidu = defInduvidu.sort_values(by='fitness', ascending = 0).reset_index(drop=1)

defInduvidu = defInduvidu[:populasiSize]
print(defInduvidu)


print('KROMOSOM TERBAIK : \n')
bestKromosom = {'THETA' : ['theta'+str(i) for i in range(genSize)],
                 'NILAI' : [defInduvidu.at[0, 'theta'+str(j)] for j in range(genSize)]
                }

bk = pd.DataFrame(bestKromosom, columns=['THETA', 'NILAI'])
print(bk)


def fungsiFinal(bestKromosom):
    harga = 0
    idx = len(data)-1
    dataHarga = pd.DataFrame(data, columns= ['open;'])
    print(' \n')
    print('Harga Saham saat ini : \n')
    for i in range(genSize):
        harga = harga + ( bk.iat[i,1] * data.at[idx-i, 'open;'] )
    return harga

print(fungsiFinal(bestKromosom))

UPDATE NEW GEN : 

      theta0    theta1    theta2  ...    theta8    theta9   fitness
0   0.132635  0.299089  0.376989  ...  0.296380  0.851855  0.003024
1   0.638300  0.532061  0.578676  ...  0.455017  0.996406  0.003024
2   0.886955  0.076464  0.083607  ...  0.755421  0.192459  0.002976
3   0.413512  0.500878  0.123901  ...  0.103730  0.051703  0.002976
4   0.892566  0.643973  0.403215  ...  0.043783  0.580943  0.002825
..       ...       ...       ...  ...       ...       ...       ...
95  0.457590  0.501923  0.230257  ...  0.296380  0.851855  0.000247
96  0.560522  0.750572  0.352579  ...  0.468367  0.325252  0.000246
97  0.892566  0.643973  0.403215  ...  0.604391  0.989568  0.000246
98  0.892566  0.643973  0.423412  ...  0.306863  0.697412  0.000242
99  0.580271  0.514746  0.271306  ...  0.246151  0.150542  0.000242

[100 rows x 11 columns]
KROMOSOM TERBAIK : 

    THETA     NILAI
0  theta0  0.132635
1  theta1  0.299089
2  theta2  0.376989
3  theta3  0.263197
4  theta4  0.240576